In [ ]:

# --- ONE-CELL DASHBOARD (Image widget; reactive sliders) ---
from pathlib import Path
import io
import numpy as np, pandas as pd
import ipywidgets as W
from ipywidgets import Layout
from IPython.display import display
import matplotlib
matplotlib.use("agg")  # render offscreen to PNG
import matplotlib.pyplot as plt

# ---------------- Paths & Safe Loading ----------------
HERE = Path(".").resolve()
DATA = HERE / "data"   # Expect meta.csv, tuning_curves.npz, psth.npz
PKG = str(DATA)        # Back-compat, if older code referenced PKG

def _demo():
    angles = np.linspace(0, 350, 36)
    time   = np.linspace(-0.5, 1.0, 151)
    rng = np.random.default_rng(0)
    meta = pd.DataFrame({
        "unit_id": np.arange(1, 31),
        "layer": rng.choice(["SG","G","IG"], size=30),
        "OSI": np.round(rng.uniform(0.1, 0.95, size=30), 3),
        "selectivity": np.round(rng.uniform(0.4, 1.1, size=30), 3),
    })
    tc = []
    for i in range(30):
        peak = rng.choice(angles)
        curve = 5 + 15*np.exp(-0.5*((angles-peak+360)%360/30)**2) + rng.normal(0,0.5,size=len(angles))
        tc.append(np.clip(curve, 0, None))
    tun_curves = np.stack(tc, axis=0)
    pc = []
    for i in range(30):
        mu = rng.uniform(0.2, 0.7)
        curve = 6 + 20*np.exp(-0.5*((time-mu)/0.12)**2) + rng.normal(0,0.8,size=len(time))
        pc.append(np.clip(curve, 0, None))
    psth_curves = np.stack(pc, axis=0)
    return meta, angles, tun_curves, time, psth_curves

def _load():
    try:
        meta = pd.read_csv(DATA / "meta.csv")
        tun  = np.load(DATA / "tuning_curves.npz")
        psth = np.load(DATA / "psth.npz")
        angles      = tun.get("angles", None)
        tun_curves  = tun.get("curves", None)
        time        = psth.get("t", None)
        psth_curves = psth.get("curves", None)
        if meta is None or angles is None or tun_curves is None or time is None or psth_curves is None:
            raise ValueError("Missing arrays")
        if len(meta) == 0 or tun_curves.shape[0] == 0 or psth_curves.shape[0] == 0:
            raise ValueError("Empty arrays")
        for col, default in [("layer", "G"), ("OSI", 0.5), ("selectivity", 0.8), ("unit_id", None)]:
            if col not in meta.columns:
                meta[col] = default if default is not None else np.arange(1, len(meta)+1)
        meta["OSI"] = pd.to_numeric(meta["OSI"], errors="coerce").fillna(0.5).clip(0, 1)
        meta["selectivity"] = pd.to_numeric(meta["selectivity"], errors="coerce").fillna(0.8).clip(0, 1.2)
        if len(meta) < 1:
            raise ValueError("No rows after coercion")
        return meta, angles, tun_curves, time, psth_curves
    except Exception:
        return _demo()

meta, angles, tun_curves, time, psth_curves = _load()

# Align sizes if mismatch
n_units = len(meta)
m = min(n_units, tun_curves.shape[0], psth_curves.shape[0])
meta = meta.iloc[:m].reset_index(drop=True)
tun_curves = tun_curves[:m]
psth_curves = psth_curves[:m]

# ---------------- Widgets & Layout ----------------
layers = sorted(pd.Series(meta["layer"]).dropna().astype(str).unique().tolist()) or ["SG","G","IG"]

def _finite_or(default, x):
    try:
        fx = float(x)
        return fx if np.isfinite(fx) else default
    except Exception:
        return default

osi_min = _finite_or(0.0, np.nanmin(meta["OSI"]) if len(meta) else 0.0)
osi_max = _finite_or(1.0, np.nanmax(meta["OSI"]) if len(meta) else 1.0)
sel_min = _finite_or(0.0, np.nanmin(meta["selectivity"]) if len(meta) else 0.0)
sel_max = _finite_or(1.2, np.nanmax(meta["selectivity"]) if len(meta) else 1.2)

layer_w = W.SelectMultiple(options=layers, value=tuple(layers), description="Layer",
                           layout=Layout(width="260px", height="120px"))
osi_w   = W.FloatRangeSlider(description="OSI range", min=0, max=1, step=0.01,
                             value=[max(0, osi_min), min(1, osi_max)], readout_format=".2f",
                             layout=Layout(width="260px"))
sel_w   = W.FloatRangeSlider(description="Sel range", min=0, max=1.2, step=0.01,
                             value=[max(0, sel_min), min(1.2, sel_max)], readout_format=".2f",
                             layout=Layout(width="260px"))
plot_w  = W.ToggleButtons(options=["Tuning","PSTH"], value="Tuning", description="Plot",
                          layout=Layout(width="260px"))

max_idx = max(len(meta)-1, 0)
idx_w   = W.IntSlider(description="Unit index", min=0, max=max_idx, step=1, value=0,
                      continuous_update=False, layout=Layout(width="260px"))

controls = W.VBox([layer_w, osi_w, sel_w, plot_w, idx_w],
                  layout=Layout(width="280px", min_width="280px"))

# Image widget to render PNG plots (no overlay)
img = W.Image(format='png', layout=Layout(width="100%", height="640px", border="1px solid #eee"))
status = W.HTML(layout=Layout(margin="6px 0 0 6px"))

right = W.VBox([img, status], layout=Layout(width="100%"))

ui = W.HBox([controls, right], layout=Layout(align_items="flex-start", width="100%"))
display(ui)

# ---------------- Rendering ----------------
def filtered_indices():
    df = meta.copy()
    if layer_w.value:
        df = df[df["layer"].astype(str).isin(list(layer_w.value))]
    lo, hi = osi_w.value
    df = df[(df["OSI"] >= lo) & (df["OSI"] <= hi)]
    lo2, hi2 = sel_w.value
    df = df[(df["selectivity"] >= lo2) & (df["selectivity"] <= hi2)]
    return df.index.to_numpy(), df

def draw_png(fig):
    bio = io.BytesIO()
    fig.savefig(bio, format="png", bbox_inches="tight", dpi=110)
    plt.close(fig)
    return bio.getvalue()

def render(*_):
    idxs, df = filtered_indices()
    n = len(df)
    if n == 0:
        # blank image
        fig = plt.figure(figsize=(8.5,5.0), dpi=110)
        plt.text(0.5, 0.5, "No units match the filters.", ha="center", va="center")
        plt.axis("off")
        img.value = draw_png(fig)
        status.value = "No matching units."
        idx_w.max = 0
        idx_w.value = 0
        return

    # Rebind slider max to number of filtered rows
    new_max = n - 1
    if idx_w.max != new_max:
        idx_w.max = new_max
        idx_w.value = min(idx_w.value, new_max)

    pos = int(np.clip(idx_w.value, 0, new_max))
    unit_global_idx = int(idxs[pos])
    row = meta.iloc[unit_global_idx]

    fig = plt.figure(figsize=(8.5,5.0), dpi=110)
    plt.grid(True, alpha=0.25)
    if plot_w.value == "Tuning":
        plt.plot(angles, tun_curves[unit_global_idx], marker="o")
        plt.title(f"Tuning — unit {int(row.get('unit_id', unit_global_idx+1))} | layer={row['layer']} | OSI={float(row['OSI']):.2f}")
        plt.xlabel("Orientation (deg)"); plt.ylabel("Response (a.u.)")
    else:
        plt.plot(time, psth_curves[unit_global_idx])
        plt.title(f"PSTH — unit {int(row.get('unit_id', unit_global_idx+1))} | layer={row['layer']} | OSI={float(row['OSI']):.2f}")
        plt.xlabel("Time (s)"); plt.ylabel("Rate (Hz)")
    plt.tight_layout()

    img.value = draw_png(fig)
    status.value = f"{n} units match filters. Showing {pos+1}/{n} (global idx {unit_global_idx})."

# Wire events
for w in (layer_w, osi_w, sel_w, plot_w, idx_w):
    w.observe(render, names="value")

render()
